### ⑦ 다음 단계를 따라 moons 데이터셋에 결정 트리를 훈련시키고 세밀하게 튜닝해보세요.

Hint여러 가지 max_leaf_nodes 값을 시도해보세요

a.  make_moons(n_samples=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [2]:
from sklearn.datasets import make_moons
X_moons, y_moons = make_moons(n_samples = 10000, noise = 0.4)

In [4]:
X_moons

array([[-0.59825271, -0.02209589],
       [ 0.71790981,  1.23311102],
       [ 1.35654849,  0.36065426],
       ...,
       [-1.03032535,  1.10402296],
       [-0.46112988,  0.25877011],
       [-1.1071793 ,  1.15573165]])

In [5]:
y_moons

array([0, 0, 1, ..., 0, 0, 0])

b.  이를 train_test_split()을 사용해 훈련 세트와 테스트 세트로 나눕니다.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size = 0.2, random_state=42)

c.  DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 서치를 수행합니다 (GridSearchCV를 사용하면 됩니다 ).

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth' : list(range(1, 7)),
    'min_samples_split' : [2, 3, 4]
    }

gs_cv = GridSearchCV(DecisionTreeClassifier(random_state=42),
                              params, cv = 3)

gs_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [9]:
gs_cv.best_estimator_

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=21, random_state=42)

d.  찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능을 측정합니다. 대략 85~87%의 정확도가 나올 것입니다.

In [10]:
from sklearn.metrics import accuracy_score

y_pred = gs_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8665

### ⑧ 다음 단계를 따라 랜덤 포레스트를 만들어보세요.

#### 랜덤 포레스트(Random Forest) 모델 구조
랜덤 포레스트(Random Forest)는 **여러 개의 결정 트리(Decision Tree)를 조합**하여 예측을 수행하는 앙상블 학습(Ensemble Learning) 모델


- 여러 개의 결정 트리를 학습하여 최종 예측
    - 분류: **투표(Voting)**
    - 회귀: **평균(Averaging)**
- 각각의 트리는 서로 다른 **훈련 데이터 샘플과 특성(Feature) 서브셋**을 사용하여 학습
- 과적합 방지, 일반화 성능 향상
    - Bagging: 중복 허용하여 랜덤 샘플 뽑아 훈련 세트 생성
    - 랜덤 feature 선택: 각 특성에서 일부 특성만 랜덤하게 선택하여 학습

- ```rf.estimators_```: 각 결정 트리 모델 저장 → 개별 트리 확인 가능


| 하이퍼파라미터 | 설명 | 추천 값 |
|--------------|--------------------------------|--------|
| `n_estimators` | 트리 개수 | 100~500 |
| `max_depth` | 트리 최대 깊이 (과적합 방지) | `None`(기본) or `10~30` |
| `max_features` | 노드 분할 시 각 트리가 사용할 특성 개수 | `sqrt` (분류) / `log2` (회귀) |
| `bootstrap` | 배깅(Bagging) 사용 여부 | `True` |
| `min_samples_split` | 노드 분할 최소 샘플 수 | `2~10` |
| `min_samples_leaf` | 리프 노드의 최소 샘플 수 | `1~5` |
| `max_leaf_nodes` | 결정 트리의 최대 리프 노드 수 | `None`(기본)→ 과적합 가능성 증가 |
| `random_state` | 랜덤 시드 | 고정 (재현성 보장) |


a.  이전 연습문제에 이어서 훈련 세트의 서브셋을 1,000개 생성합니다. 각각은 랜덤으로 선택된 100개의 샘플을 담고 있습니다.      

Hint사이킷런의 ShuffleSplit을 사용할 수 있습니다

- ```ShuffleSplit()```
    - ```n_splits```: 서브셋 개수
    - ```train_size```: 서브셋에 포함될 데이터 샘플 수  
    ( = 훈련 데이터의 크기)
    - ```test_size```: 테스트 데이터의 크기  
    만약 test_size = 100으로 설정 시, 자동으로 train_size = 전체 데이터 - 100



In [12]:
from sklearn.model_selection import ShuffleSplit

# 서브셋 개수
subset = 1000
# 각 서브셋 크기
n_samples = 100

rs = ShuffleSplit(n_splits = subset, train_size = n_samples,random_state = 42)

# 서브셋 저장 리스트
mini_sets = []

# ShuffleSplit을 이용한 서브셋 생성
for mini_train_index, _ in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

# print(f"총 {len(mini_sets)}개의 서브셋 생성")  # 1000개 확인
# print(f"각 서브셋의 크기: {mini_sets[0][0].shape}")  # (100, 2)

총 1000개의 서브셋 생성
각 서브셋의 크기: (100, 2)


b.  이전 연습문제에서 찾은 최적의 매개변수를 사용해 각 서브셋에 결정 트리를 훈련시킵니다. 테스트 세트로 이 1,000개의 결정 트리를 평가합니다. 더 작은 데이터셋에서 훈련되었기 때문에 이 결정 트리는 앞서 만든 결정 트리보다 성능이 떨어져 약 80%의 정확도를 냅니다.

In [16]:
from sklearn.base import clone
import numpy as np

# 1000개(서브셋 개수만큼)의 결정트리 생성
forest = [clone(gs_cv.best_estimator_) for _ in range(subset)]

accuracy_scores = []

# 서브셋에 대해 이전 결정트리 훈련
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# 정확도 계산
np.mean(accuracy_scores)

np.float64(0.8029014999999999)

c.  이제 마술을 부릴 차례입니다. 각 테스트 세트 샘플에 대해 1,000개의 결정 트리 예측을 만들고 다수로 나온 예측만 취합니다(사이파이의 mode() 함수를 사용할 수 있습니다). 그러면 테스트 세트에 대한 다수결 예측 majority-vote prediction이 만들어집니다.

In [19]:
y_pred = np.empty([subset, len(X_test)])

for tree_idx, tree in enumerate(forest):
    y_pred[tree_idx] = tree.predict(X_test)

y_pred

array([[1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 0., 0., 1.],
       ...,
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 0., 1.]])

In [20]:
from scipy.stats import mode

majority_vote_prediction, n_votes = mode(y_pred, axis = 0, keepdims = False)

In [24]:
len(X_test)

2000

In [21]:
majority_vote_prediction.shape, n_votes.shape

((2000,), (2000,))

In [22]:
majority_vote_prediction

array([1., 1., 1., ..., 1., 0., 1.])

In [23]:
n_votes

array([962., 951., 960., ..., 760., 909., 957.])

d  테스트 세트에서 이 예측을 평가합니다. 앞서 만든 모델보다 조금 높은 (약 0.5~1.5% 정도 ) 정확도를 얻게 될 것입니다. 축하합니다, 랜덤 포레스트 분류기를 훈련시켰습 니다

In [25]:
accuracy_score(y_test, majority_vote_prediction)

0.8725

In [34]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators = 1000,
    max_depth = 6,
    max_features = 'sqrt',
    max_leaf_nodes = 21,
    bootstrap = True,
    random_state = 42
)

rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

In [35]:
accuracy_score(y_test, y_pred_rf)

0.874